<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Quantitative Finance

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [training@tpq.io](mailto:trainin@tpq.io) | [@dyjh](http://twitter.com/dyjh)

# Performance Python

## Binomial Trees

### Python

In [ ]:
import math
import numba
import numpy as np

In [ ]:
S0 = 36.  
T = 1.0  
r = 0.06  
sigma = 0.2  

In [ ]:
def simulate_tree(M):
    dt = T / M  
    u = math.exp(sigma * math.sqrt(dt))  
    d = 1 / u  
    S = np.zeros((M + 1, M + 1))
    S[0, 0] = S0
    z = 1
    for t in range(1, M + 1):
        for i in range(z):
            S[i, t] = S[i, t-1] * u
            S[i+1, t] = S[i, t-1] * d
        z += 1
    return S

In [ ]:
np.set_printoptions(formatter={'float':
                               lambda x: '%7.2f' % x})  

In [ ]:
simulate_tree(4)  

In [ ]:
%time simulate_tree(500)  

### NumPy

In [ ]:
M = 4

In [ ]:
up = np.arange(M + 1)
up = np.resize(up, (M + 1, M + 1))  
up

In [ ]:
down = up.T * 2  
down

In [ ]:
up - down  

In [ ]:
dt = T / M

In [ ]:
S0 * np.exp(sigma * math.sqrt(dt) * (up - down))  

In [ ]:
def simulate_tree_np(M):
    dt = T / M
    up = np.arange(M + 1)
    up = np.resize(up, (M + 1, M + 1))
    down = up.transpose() * 2
    S = S0 * np.exp(sigma * math.sqrt(dt) * (up - down))
    return S

In [ ]:
simulate_tree_np(4)

In [ ]:
%time simulate_tree_np(500)

### Numba

In [ ]:
simulate_tree_nb = numba.jit(simulate_tree)

In [ ]:
simulate_tree_nb(4)

In [ ]:
%time simulate_tree_nb(500)

In [ ]:
%timeit simulate_tree_nb(500)

### Cython

In [ ]:
import cython

In [ ]:
%load_ext Cython

In [ ]:
%%cython -a
import numpy as np
cimport cython
from libc.math cimport exp, sqrt
cdef float S0 = 36.
cdef float T = 1.0
cdef float r = 0.06
cdef float sigma = 0.2
def simulate_tree_cy(int M):
    cdef int z, t, i
    cdef float dt, u, d
    cdef float[:, :] S = np.zeros((M + 1, M + 1),
                                  dtype=np.float32)  
    dt = T / M
    u = exp(sigma * sqrt(dt))
    d = 1 / u
    S[0, 0] = S0
    z = 1
    for t in range(1, M + 1):
        for i in range(z):
            S[i, t] = S[i, t-1] * u
            S[i+1, t] = S[i, t-1] * d
        z += 1
    return np.array(S)

In [ ]:
simulate_tree_cy(4)

In [ ]:
%time simulate_tree_cy(500)

In [ ]:
%timeit S = simulate_tree_cy(500)

## Multiprocessing

In [ ]:
import multiprocess as mp

In [ ]:
# on Google Colab import:
# import multiprocessing as mp

In [ ]:
N, M = 50, 500

In [ ]:
func = simulate_tree

In [ ]:
%%time
trees = list() 
for _ in range(N):
    trees.append(func(M))

In [ ]:
pool = mp.Pool(processes=8)  

In [ ]:
%time trees = pool.map(func, N * [M])

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>